### 2) Написать программу, которая собирает «Хиты продаж» с сайтов техники mvideo, onlinetrade и складывает данные в БД. Магазины можно выбрать свои. Главный критерий выбора: динамически загружаемые товары

In [1]:
from selenium import webdriver          #Основной элемент
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait   # Select - Поле множественного выбора
from selenium.webdriver.common.by import By  #используем Селениум
from selenium.webdriver.common.keys import Keys    #Клавиши клавиатуры
from selenium.webdriver.common.action_chains import ActionChains
import pprint
from pymongo import MongoClient
import time
import datetime
import re

In [2]:
link_mvideo = "https://www.mvideo.ru/"
link_onlinetrade = ""

In [3]:
driver = webdriver.Chrome()   #Движок Хрома
driver.maximize_window()
driver.get(link_mvideo)

In [4]:
WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "flocktory-widget")))
flocktories_iframes = driver.find_elements_by_class_name("flocktory-widget")
len(flocktories_iframes)

3

In [5]:
for iframe in flocktories_iframes:
    driver.switch_to.frame(iframe)
    try:
        close_btn = driver.find_element_by_xpath("//div[contains(@data-fl-track, 'click-close-x')]")
        close_btn.click()
        hits = iframe
    except Exception as e:
        pass
    driver.switch_to.default_content()

In [6]:
ajaxs = driver.find_elements_by_xpath("//div[contains(@data-init, 'ajax-category-carousel')]")
len(ajaxs)

3

In [7]:
for elem in ajaxs:
    if "Хиты продаж" in elem.get_attribute('innerHTML'):
        hits = elem
        break

In [8]:
def get_item_info(one_item):
    try:
        item_name = one_item.find_element_by_class_name("sel-product-tile-title").text
        item_href = one_item.find_element_by_class_name("sel-product-tile-title").get_attribute("href")
        item_price = int(re.sub("\D", "", one_item.find_element_by_class_name("c-pdp-price__current").text))
        return {"name": item_name,"href": item_href, "price": item_price}
    except:
        return {}

In [9]:
def wait_for_ajax(driver):
    wait = WebDriverWait(driver, 15)
    try:
        wait.until(lambda driver: driver.execute_script('return jQuery.active') == 0)
        wait.until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
    except Exception as e:
        pass

In [10]:
next_btn = hits.find_element_by_class_name("sel-hits-button-next")
prev_btn = hits.find_element_by_class_name("sel-hits-button-prev")
action = webdriver.ActionChains(driver)
action.move_to_element(hits).perform()
while True:
    next_btn.click()
    wait_for_ajax(driver)
    try:
        hits.find_element_by_class_name("disabled")
        break
    except Exception as e:
        pass

In [11]:
items_info = []
for i in range(5):
    hits_list = hits.find_elements_by_class_name("gallery-list-item")
    for item in hits_list:
        try:
            item_info = get_item_info(item)
            if item_info != {} and item_info not in items_info:
                items_info.append(item_info)
        except Exception as e:
            pass
    prev_btn.click()
    time.sleep(1)

In [12]:
for item in items_info:
    print(item['name'], item['price'], item['href'], "", sep="\n")

Видеокамера экшн SJCAM SJ4000 Air Black
3990
https://www.mvideo.ru/products/videokamera-ekshn-sjcam-sj4000-air-black-10020390

Музыкальная система Midi LG XBOOM OK85
23990
https://www.mvideo.ru/products/muzykalnaya-sistema-midi-lg-xboom-ok85-10018348

Спортивные наушники Bluetooth Honor Sport AM61 Red
2190
https://www.mvideo.ru/products/sportivnye-naushniki-bluetooth-honor-sport-am61-red-50110516

Утюг Philips GC4537/70
3990
https://www.mvideo.ru/products/utug-philips-gc4537-70-20054614

Наушники Bluetooth JBL Live 500BT Black
5490
https://www.mvideo.ru/products/naushniki-bluetooth-jbl-live-500bt-black-50125459

Погружной блендер Redmond RHB-2946
3990
https://www.mvideo.ru/products/pogruzhnoi-blender-redmond-rhb-2946-20042933

Ноутбук Lenovo IdeaPad 330-15IKB (81DC0107RU)
39990
https://www.mvideo.ru/products/noutbuk-lenovo-ideapad-330-15ikb-81dc0107ru-30042205

Пылесос с контейнером для пыли Bosch GS 10 BGS1U1805
6990
https://www.mvideo.ru/products/pylesos-s-konteinerom-dlya-pyli-bosch

In [13]:
driver.quit()

In [14]:
mongoclient = MongoClient('mongodb://127.0.0.1:27017')
db = mongoclient['goods']
goodsdb = db.goods
mongoclient.drop_database("goods")

In [15]:
goodsdb.insert_many(items_info)

In [16]:
def show_what_in_database(print_limit=0):
    cursor = goodsdb.find()
    i = 0
    for document in cursor:
        pprint.pprint(document)
        print()
        i += 1
        if print_limit != 0:
            if i == print_limit:
                return

In [17]:
show_what_in_database()

{'_id': ObjectId('5d2ca423daf041724b31806a'),
 'href': 'https://www.mvideo.ru/products/videokamera-ekshn-sjcam-sj4000-air-black-10020390',
 'name': 'Видеокамера экшн SJCAM SJ4000 Air Black',
 'price': 3990}

{'_id': ObjectId('5d2ca423daf041724b31806b'),
 'href': 'https://www.mvideo.ru/products/muzykalnaya-sistema-midi-lg-xboom-ok85-10018348',
 'name': 'Музыкальная система Midi LG XBOOM OK85',
 'price': 23990}

{'_id': ObjectId('5d2ca423daf041724b31806c'),
 'href': 'https://www.mvideo.ru/products/sportivnye-naushniki-bluetooth-honor-sport-am61-red-50110516',
 'name': 'Спортивные наушники Bluetooth Honor Sport AM61 Red',
 'price': 2190}

{'_id': ObjectId('5d2ca423daf041724b31806d'),
 'href': 'https://www.mvideo.ru/products/utug-philips-gc4537-70-20054614',
 'name': 'Утюг Philips GC4537/70',
 'price': 3990}

{'_id': ObjectId('5d2ca423daf041724b31806e'),
 'href': 'https://www.mvideo.ru/products/naushniki-bluetooth-jbl-live-500bt-black-50125459',
 'name': 'Наушники Bluetooth JBL Live 500BT 